<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/curso-ml-espanol/blob/master/C1%20-%20Aprendizaje%20Supervisado/W1/C1_W1_Lab03_Funcion_Costo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/curso-ml-espanol.git

%cd curso-ml-espanol/C1 - Aprendizaje Supervisado/W1/

!cp -r deeplearning.mplstyle /content/

!cp -r images /content/

!cp -r lab_utils_uni.py /content/

%cd /content/

!rm -rf curso-ml-espanol/

# Lab: Función de Costo (Cost Funcion)
<figure>
    <center> <img src="./images/C1_W1_L3_S2_Lecture_b.jpeg"  style="width:1000px;height:200px;" ></center>
</figure>



## Objetivos
En este laboratorio:
- Implementarás y explorarás la función de costo para la regresión lineal con una variable.


## Herramientas
En este laboratorio haremos uso de:
- NumPy, una biblioteca popular para el cálculo científico
- Matplotlib, una biblioteca popular para la visualización de datos
- rutinas locales de visualización en el archivo lab_utils_uni.py en el directorio local

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_uni import plt_intuition, plt_stationary, plt_update_onclick, soup_bowl
plt.style.use('./deeplearning.mplstyle')

## Declaración del Problema

Crear un modelo que pueda predecir los precios de las viviendas dada el tamaño de la casa.  
Usaremos los mismos dos puntos de datos que antes del laboratorio anterior: una casa de 1000 pies cuadrados se vendió por $300,000 y una casa de 2000 pies cuadrados se vendió por $500,000.


| Tamaño (1000 pies cuadrados) | Precio (miles de dólares) |
| -----------------------------| -------------------------- |
| 1                            | 300                        |
| 2                            | 500                        |

In [ ]:
x_train = np.array([1.0, 2.0])           #(tamaño en 1000 pies cuadrados)
y_train = np.array([300.0, 500.0])           #(precio en miles de dólares)

## Cálculo del Costo
El término 'costo' en esta asignación podría ser un poco confuso ya que los datos se refieren al costo de la vivienda. Aquí, el costo es una medida de qué tan bien nuestro modelo está prediciendo el precio objetivo de la casa. El término 'precio' se utiliza para los datos de vivienda.

La ecuación para el costo con una variable es:
  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{1}$$ 
 
donde 
  $$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{2}$$
  
- $f_{w,b}(x^{(i)})$ es nuestra predicción para el ejemplo $i$ usando los parámetros $w,b$.  
- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ es la diferencia al cuadrado entre el valor objetivo y la predicción.   
- Estas diferencias se suman sobre todos los $m$ ejemplos y se dividen por `2m` para producir el costo, $J(w,b)$.  
>Nota, en las clases los rangos de sumatoria típicamente van de 1 a m, mientras que en el código van de 0 a m-1.

El código a continuación calcula el costo recorriendo cada ejemplo. En cada ciclo:
- `f_wb`, se calcula una predicción
- se calcula la diferencia entre el objetivo y la predicción y se eleva al cuadrado.
- esto se suma al costo total.

In [ ]:
def compute_cost(x, y, w, b): 
    """
    Calcula la función de costo para la regresión lineal.
    
    Argumentos:
      x (ndarray (m,)): Datos, m ejemplos 
      y (ndarray (m,)): valores objetivos
      w,b (escalar)    : parámetros del modelo  
    
    Retorna:
        total_cost (float): El costo de usar w,b como los parámetros para la regresión lineal
               para ajustar los puntos de datos en x y y
    """
    # número de ejemplos de entrenamiento
    m = x.shape[0] 
    
    suma_costo = 0 
    for i in range(m): 
        f_wb = w * x[i] + b   
        costo = (f_wb - y[i]) ** 2  
        suma_costo = suma_costo + costo  
    costo_total = (1 / (2 * m)) * suma_costo  

    return costo_total

## Intuición de la Función de Costo

<img align="left" src="./images/C1_W1_Lab02_GoalOfRegression.jpeg"    style=" width:380px; padding: 10px;  " /> 

Tu objetivo es encontrar un modelo $f_{w,b}(x) = wx + b$, con parámetros $w,b$, que prediga con precisión los valores de las casas dado un input $x$. El costo es una medida de cuán preciso es el modelo en los datos de entrenamiento.

La ecuación de costo (1) arriba muestra que si se pueden seleccionar $w$ y $b$ de tal manera que las predicciones $f_{w,b}(x)$ coincidan con los datos objetivos $y$, el término $(f_{w,b}(x^{(i)}) - y^{(i)})^2 $ será cero y el costo minimizado. ¡En este simple ejemplo de dos puntos, puedes lograr esto!

En el laboratorio anterior, determinaste que $b=100$ proporcionaba una solución óptima, así que establezcamos $b$ en 100 y enfoquémonos en $w$.

A continuación, usa el control deslizante para seleccionar el valor de $w$ que minimiza el costo. Puede tardar unos segundos en actualizarse el gráfico.

In [ ]:
plt_intuition(x_train,y_train)

El gráfico contiene algunos puntos que vale la pena mencionar.

- el costo se minimiza cuando $w = 200$, lo que coincide con los resultados del laboratorio anterior
- Debido a que la diferencia entre el objetivo y la predicción se eleva al cuadrado en la ecuación de costo, el costo aumenta rápidamente cuando $w$ es demasiado grande o demasiado pequeño.
- Usar el `w` y el `b` seleccionados minimizando el costo resulta en una línea que se ajusta perfectamente a los datos.

## Visualización de la Función de Costo - 3D

Puedes ver cómo varía el costo con respecto a *ambos*, `w` y `b`, trazando en 3D o utilizando un gráfico de contorno.
Vale la pena mencionar que algunas de las tramas en este curso pueden volverse bastante complejas. Las rutinas de trazado se proporcionan y, aunque puede ser instructivo leer el código para familiarizarse con los métodos, no es necesario para completar el curso con éxito. Las rutinas están en lab_utils_uni.py en el directorio local.

### Conjunto de Datos Más Grande
Es instructivo ver un escenario con algunos puntos de datos adicionales. Este conjunto de datos incluye puntos que no caen en la misma línea. ¿Qué significa eso para la ecuación de costo? ¿Podemos encontrar $w$ y $b$ que nos den un costo de 0?

In [ ]:
x_train = np.array([1.0, 1.7, 2.0, 2.5, 3.0, 3.2])
y_train = np.array([250, 300, 480,  430,   630, 730,])

En el gráfico de contorno, haz clic en un punto para seleccionar `w` y `b` para lograr el costo más bajo. Usa los contornos para guiar tus selecciones. Ten en cuenta que puede tardar unos segundos en actualizar el gráfico.

In [ ]:
plt.close('all') 
fig, ax, dyn_items = plt_stationary(x_train, y_train)
updater = plt_update_onclick(fig, ax, x_train, y_train, dyn_items)

Arriba, observa las líneas punteadas en el gráfico de la izquierda. Estas representan la porción del costo contribuida por cada ejemplo en tu training set. En este caso, valores de aproximadamente $w=209$ y $b=2.4$ proporcionan un costo bajo. Ten en cuenta que, como nuestros ejemplos de entrenamiento no están en una línea, el costo mínimo no es cero.

### Superficie de Costo Convexo
El hecho de que la función de costo eleve al cuadrado la pérdida asegura que la 'superficie de error' sea convexa, como un tazón de sopa. Siempre tendrá un mínimo que se puede alcanzar siguiendo el gradiente en todas las dimensiones. En la trama anterior, debido a que las dimensiones de $w$ y $b$ escalan de manera diferente, esto no es fácil de reconocer. La siguiente trama, donde $w$ y $b$ son simétricos, se mostró en la clase:

In [ ]:
soup_bowl()

# ¡Felicidades!
Has aprendido lo siguiente:
 - La ecuación de costo proporciona una medida de qué tan bien tus predicciones coinciden con tus datos de entrenamiento.
 - Minimizar el costo puede proporcionar valores óptimos de $w$, $b$.